In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import numpy as np

# 連接到資料庫
conn = sqlite3.connect('/content/drive/MyDrive/anuenews/anuenews.db')
# cursor object
cursor = conn.cursor()


uniqueSql = '''select * from news t'''


res = requests.get('https://news.cnyes.com/news/cat/tw_bull?exp=a')
soup = BeautifulSoup(res.text,"html.parser")


titletime = []
id = []
content = []
  

# 日期============================
times = soup.select('._67tN')
for time in times:
  head,mid,tail = time.find('time')['datetime'].partition('T') # partition‘T'之後分為頭部，中部，尾部三個
  titletime.append(head)
# print(titletime)

# 內容============================
titles = soup.find_all('h3')
for title in titles:
  head,mid,tail = title.text.partition(':')
  id.append(head)
  content.append(tail)
# print(content)


data = {
  'date': titletime,
  'code': id,
  'content': content
}

df = pd.DataFrame(data)
# print(df)

# 今日新闻
newsToday = pd.Timestamp.today().strftime("%Y-%m-%d") # 格式化日期
df_mask = df['date']==newsToday            # 日期為今天的存檔
positions = np.flatnonzero(df_mask)
filtered_df=df.iloc[positions]


# for index, row in df.iterrows():
#     print(row['date'], row['code'],row['content'])

for index, row in filtered_df.iterrows():
  try:
    cursor.execute(
        """INSERT INTO news 
          (date, code, content)
          values(?,?,?)""",
          (row['date'],
          row['code'],
          row['content']),
        )
    conn.commit()
    # 現在看來，commit()方法需要跟在增（insert）、刪（delete）、改（update）、查（select）的任何execute()語句後面。commit是把查詢語句提交到資料庫內，而不只是要向資料庫提交增、添的資料。
  except:
    pass

pd.read_sql(uniqueSql,conn)

In [ ]:
# 导出为csv
import pandas as pd
import numpy as np
import sqlite3


conn2 = sqlite3.connect('anuenews.db')
uniqueSql2 = '''select * from news t'''

df2 = pd.read_sql(uniqueSql2,conn2)
df3 = pd.DataFrame(df2, columns=['code','content','date'])

# 今日新闻
newsToday = pd.Timestamp.today().strftime("%Y-%m-%d") # 格式化日期
# df_mask = df3['date']==newsToday            # 日期為今天的存檔
df_mask = df3['date']=='2022-08-25'           # 日期為今天的存檔
positions = np.flatnonzero(df_mask)
filtered_df=df3.iloc[positions]


filtered_df.to_csv("鉅亨每日新聞.csv", encoding="big5hkscs",index=False)
print('導出成功')

In [ ]:
# 创建资料库 ====================================================================
# import sqlite3


# # connecting to sqlite
# conn = sqlite3.connect('anuenews.db')
# # cursor object
# cursor = conn.cursor()

# # Creating table
# # 數據庫形態+content內容為不可重複
# news = """ CREATE TABLE news (
#     date DATE,
#     code TEXT,
#     content TEXT,
#     UNIQUE(content)
# ); """

# cursor.execute(news)
# ================================================================================

In [ ]:
# 删除资料 ====================================================================
# conn = sqlite3.connect('anuenews.db')
# cursor = conn.cursor()
# sql_delete_query = """DELETE from news where code = '〈美股早盤〉市場靜待Fed升息線索 主要指數開低'"""
# cursor.execute(sql_delete_query)
# conn.commit()
# pd.read_sql(uniqueSql,conn)
# ==============================================================================